# TUTORIAL:
# Julia for technical computing

# Chapter 3. Multiple dispatch

What is multiple dispatch? **Dispatch** means that Julia will call a particular version of a function, based on the types of the arguments. **Multiple dispatch** means that Julia takes the types of all of the arguments into account, not just one. **Dynamic multiple dispatch** means that the same code will always result in the same function call, regardless of whether the Julia compiler was able to infer the types of the arguments at compile-time or had to do a check (*dynamically*) at run-time.

It is through advanced **type inference** that Julia can often skip the run-time check. If functions unambiguously map the types of their input arguments to an output type, if they are *type stable*, then Julia can follow the flow of types through your code without actually executing the code.

Of course we aim to avoid runtime type checks, because they slow the execution of our code. But we are not fanatic about it either. After all, **an interactive and dynamic programming language is mostly about convenience**. We do want to optimize the time-critical path in our code and Julia allows that, without resorting to another language for the parts that have to be fast. Outside of that path, by all means, use untyped fields and violate type-stability at will! Live your life the happy I-don't-know-about-types-and-I-do-not-want-to-know way, and enjoy it.

In this chapter we set out to illustrate the possibilities enabled by multiple dispatch in Julia. You will find little in this chapter that compares to Matlab or Python. We do elaborate on the difference between multiple dispatch and OOP concepts like method overloading and polymorphism. But that is my bias. Note that Julia is neither the first nor the only language to offer multiple dispatch, or *multi-methods*. Other examples include Common Lisp and Dylan.

## 1. Dispatch and algorithm selection

### There are functions and methods. What's the difference?

I did not tell you this before, but perhaps you've noticed: there are [functions](http://julia.readthedocs.org/en/release-0.4/manual/functions/) and there are [methods](http://julia.readthedocs.org/en/release-0.4/manual/methods/) in Julia. One function can have several methods. Each method corresponds to a combination of types of arguments.

Here is an example:

In [1]:
action(x::Integer) = "Acting on Integer"

action (generic function with 1 method)

In [2]:
action(x::AbstractFloat) = "Acting on AbstractFloat"

action (generic function with 2 methods)

Julia tells me that `action` is a generic function with 2 methods. Here you see dispatch at work:

In [3]:
action(1)

"Acting on Integer"

In [4]:
action(1.0)

"Acting on AbstractFloat"

You can list all methods of a function, and you can also simply ask Julia which method it intends to invoke for a given set of arguments.

In [5]:
methods(action)

# 2 methods for generic function "action":
action(x::Integer) at In[1]:1
action(x::AbstractFloat) at In[2]:1

In [6]:
@which action(1)

action(x::Integer) at In[1]:1

In [7]:
@which action(1.0)

action(x::AbstractFloat) at In[2]:1

### Types match with their supertypes

In both methods we provided the argument had an abstract type. When Julia encounters a set of argument types and is looking for a function to dispatch to, an abstract type matches with all of its subtypes. However, if you have a specific implementation for a specific subtype, then you can provide a more specific method. Julia will dispatch to the most specific method available.

In [8]:
action(x::Int32) = "For Int32 I have a better algorithm"

action (generic function with 3 methods)

In [9]:
methods(action)

# 3 methods for generic function "action":
action(x::Int32) at In[8]:1
action(x::Integer) at In[1]:1
action(x::AbstractFloat) at In[2]:1

In [10]:
action(1)

"Acting on Integer"

In [11]:
action(Int32(1))

"For Int32 I have a better algorithm"

### Methods are algorithms are methods: Julia selects the best available algorithm for the problem at hand

Function overloading is possible in many typed languagues, but in Julia it is really pervasive. You add methods to existing functions all the time. Popular ones are the basic operators, like `+`:

In [12]:
methods(+)   # careful, long output! Scroll down afterwards

# 155 methods for generic function "+":
+(x::Bool) at bool.jl:33
+(x::Bool, y::Bool) at bool.jl:36
+(y::AbstractFloat, x::Bool) at bool.jl:46
+(x::Int64, y::Int64) at int.jl:8
+(x::Int8, y::Int8) at int.jl:16
+(x::UInt8, y::UInt8) at int.jl:16
+(x::Int16, y::Int16) at int.jl:16
+(x::UInt16, y::UInt16) at int.jl:16
+(x::Int32, y::Int32) at int.jl:16
+(x::UInt32, y::UInt32) at int.jl:16
+(x::UInt64, y::UInt64) at int.jl:16
+(x::Int128, y::Int128) at int.jl:16
+(x::UInt128, y::UInt128) at int.jl:16
+(x::Integer, y::Ptr{T}) at pointer.jl:77
+(x::Float32, y::Float32) at float.jl:203
+(x::Float64, y::Float64) at float.jl:204
+(z::Complex{T<:Real}, w::Complex{T<:Real}) at complex.jl:112
+(x::Bool, z::Complex{Bool}) at complex.jl:119
+(z::Complex{Bool}, x::Bool) at complex.jl:120
+(x::Bool, z::Complex{T<:Real}) at complex.jl:126
+(z::Complex{T<:Real}, x::Bool) at complex.jl:127
+(x::Real, z::Complex{Bool}) at complex.jl:133
+(z::Complex{Bool}, x::Real) at complex.jl:134
+(x::Real, z::Complex{T<:Real}) at complex.jl:145
+(z::Complex{T<:Real}, x::Real) at complex.jl:146
+(x::Rational{T<:Integer}, y::Rational{T<:Integer}) at rational.jl:170
+(x::Bool, A::AbstractArray{Bool,N}) at arraymath.jl:136
+(x::Integer, y::Char) at char.jl:43
+(a::Float16, b::Float16) at float16.jl:136
+(x::BigInt, y::BigInt) at gmp.jl:246
+(a::BigInt, b::BigInt, c::BigInt) at gmp.jl:269
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) at gmp.jl:275
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) at gmp.jl:282
+(x::BigInt, c::Union{UInt16,UInt32,UInt64,UInt8}) at gmp.jl:294
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigInt) at gmp.jl:298
+(x::BigInt, c::Union{Int16,Int32,Int64,Int8}) at gmp.jl:310
+(c::Union{Int16,Int32,Int64,Int8}, x::BigInt) at gmp.jl:311
+(x::BigFloat, y::BigFloat) at mpfr.jl:208
+(x::BigFloat, c::Union{UInt16,UInt32,UInt64,UInt8}) at mpfr.jl:215
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigFloat) at mpfr.jl:219
+(x::BigFloat, c::Union{Int16,Int32,Int64,Int8}) at mpfr.jl:223
+(c::Union{Int16,Int32,Int64,Int8}, x::BigFloat) at mpfr.jl:227
+(x::BigFloat, c::Union{Float16,Float32,Float64}) at mpfr.jl:231
+(c::Union{Float16,Float32,Float64}, x::BigFloat) at mpfr.jl:235
+(x::BigFloat, c::BigInt) at mpfr.jl:239
+(c::BigInt, x::BigFloat) at mpfr.jl:243
+(a::BigFloat, b::BigFloat, c::BigFloat) at mpfr.jl:379
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) at mpfr.jl:385
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) at mpfr.jl:392
+(x::Irrational{sym}, y::Irrational{sym}) at irrationals.jl:72
+(x::Number) at operators.jl:73
+{T<:Number}(x::T<:Number, y::T<:Number) at promotion.jl:211
+{T<:AbstractFloat}(x::Bool, y::T<:AbstractFloat) at bool.jl:43
+(x::Number, y::Number) at promotion.jl:167
+(r1::OrdinalRange{T,S}, r2::OrdinalRange{T,S}) at operators.jl:324
+{T<:AbstractFloat}(r1::FloatRange{T<:AbstractFloat}, r2::FloatRange{T<:AbstractFloat}) at operators.jl:331
+{T<:AbstractFloat}(r1::LinSpace{T<:AbstractFloat}, r2::LinSpace{T<:AbstractFloat}) at operators.jl:350
+(r1::Union{FloatRange{T<:AbstractFloat},LinSpace{T<:AbstractFloat},OrdinalRange{T,S}}, r2::Union{FloatRange{T<:AbstractFloat},LinSpace{T<:AbstractFloat},OrdinalRange{T,S}}) at operators.jl:363
+(x::Ptr{T}, y::Integer) at pointer.jl:75
+{S,T}(A::Range{S}, B::Range{T}) at arraymath.jl:69
+{S,T}(A::Range{S}, B::AbstractArray{T,N}) at arraymath.jl:87
+(A::BitArray{N}, B::BitArray{N}) at bitarray.jl:845
+{T}(B::BitArray{2}, J::UniformScaling{T}) at linalg/uniformscaling.jl:28
+(A::Array{T,2}, B::Diagonal{T}) at linalg/special.jl:122
+(A::Array{T,2}, B::Bidiagonal{T}) at linalg/special.jl:122
+(A::Array{T,2}, B::Tridiagonal{T}) at linalg/special.jl:122
+(A::Array{T,2}, B::SymTridiagonal{T}) at linalg/special.jl:131
+(A::Array{T,N}, B::SparseMatrixCSC{Tv,Ti<:Integer}) at sparse/sparsematrix.jl:906
+{P<:Union{Base.Dates.CompoundPeriod,Base.Dates.Period}}(x::Union{DenseArray{P<:Union{Base.Dates.CompoundPeriod,Base.Dates.Period},N},SubArray{P<:Union{Base.Dates.Compound

In my workspace at the time of writing, Julia reports that `+` has 155 methods. Each of them is optimized for a specific combination of types. You may have spotted a low-level method for the addition of two boolean values (`Bool`), as well as a high-level method for the addition of a lower-triangular and an upper-triangular matrix.
Whenever you use `+`, or whenever you invoke another function that uses `+`, a specific optimized method is used. Even better, that specific method is compiled specifically for the types you are working with, enabling all sorts of optimizations. This all happens automatically and you don't have to think about it.

We will attempt to illustrate the power of this property throughout this chapter. The selection of the best algorithm does not happen just for `+`. It happens for all functions, all the time, for low-level as well as high-level operations. A good design of Julia code makes sure that different problems map to different types, or that different properties of a problem match to different types. You write the algorithms in terms of these types, and **Julia makes sure that the right algorithm is called at the right time**. As you see, this is not entirely for free. It is not really the case that Julia selects the right algorithm. Julia selects the method that best matches the type signature. It is up to you, the programmer, to make sure that this corresponds to the right algorithm.

This does mean that **multiple dispatch affects the design of your code**. You design data structures with dispatch in mind, and you write algorithms that apply to types. This is not difficult to achieve once you're used to it, but it is quite pervasive and it is so pretty much from the start.

### An example: arithmetic modulo N

Let's add our own method to the generic function `+`. We create a type of integers modulo N and define their addition.

In the code below, we use a so-called *inner constructor*. The inner constructor is a function with the name of the type that is included in the definition of the type. The type parameter `N` is available inside the type definition. We use the constructor to make sure that field `n` is always betweeen `0` and `N-1`. Read more about constructors in the [manual](http://julia.readthedocs.org/en/release-0.4/manual/constructors/).

In [13]:
immutable ModInt{N}
    n :: Int64
    
    ModInt(n) = new(mod(n,N))  # Inner constructor: make sure that 0 <= n < N
end

We can create a `ModInt` by explicitly calling the constructor with type parameters.

In [14]:
a = ModInt{7}(2)

ModInt{7}(2)

Alternatively, if you like numbers modulo 7 very much, you can define an alias just for this case.

In [15]:
typealias ModInt7 ModInt{7}

ModInt{7}

In [16]:
ModInt7(2)

ModInt{7}(2)

Enough syntax worries, that's not our focus: let's define addition. We include the parameter `N` in the definition of `+` for `ModInt`'s, so our method of `+` will be a parametric one. Note that the type parameter `N` is the same for the types of arguments `a` and `b`. The method will only apply to `ModInt`'s with the same parameter `N`. This is called **diagonal dispatch**.

In [17]:
import Base: +
+{N}(a::ModInt{N}, b::ModInt{N}) = ModInt{N}(a.n+b.n)

+ (generic function with 156 methods)

In [18]:
a = ModInt{7}(4)
b = ModInt{7}(5)
a+b

ModInt{7}(2)

In [19]:
@which a+b

+{N}(a::ModInt{N}, b::ModInt{N}) at In[17]:2

There you are. Whenever you add two `ModInt`'s together anywhere in your code, this method will be called. Add a few other arithmetic operations, and chances are you can build matrices of `ModInt`'s and perform linear algebra operations.

Julia does not know what to do when adding `ModInt`'s with different `N`, because we did not define it. (I would not know what to do mathematically in this case either.) The type signature does not match with any of the methods of `+`.

In [20]:
ModInt{7}(2) + ModInt{5}(4)

LoadError: LoadError: MethodError: `+` has no method matching +(::ModInt{7}, ::ModInt{5})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
  +{N}(::ModInt{N}, !Matched::ModInt{N})
while loading In[20], in expression starting on line 1

Let's add another operation and see more of multiple dispatch at work. We can define multiplication by an integer:

In [25]:
import Base: *

*{N}(a::Integer, b::ModInt{N}) = ModInt{N}(a*b.n)

* (generic function with 140 methods)

In [26]:
4*ModInt{31}(12)

ModInt{31}(17)

I have typed `a` to be an `Integer`. This means that in one line I have defined multiplication (only on the left!) with `Int64`, `Int32` and so on. If, for some reason, you have a better way of interacting with `Int32`, you can make a more specific method. Even with multiple arguments, Julia will select the most specific method, favouring subtypes over supertypes.

In [27]:
function *{N}(a::Int32, b::ModInt{N})
    println("Hi. I see you're multiplying with an Int32. Good, because I have a better algorithm for this case!")
    ModInt{N}(a*b.n)
end

* (generic function with 140 methods)

In [28]:
2 * ModInt{7}(5)

ModInt{7}(3)

In [29]:
Int32(2) * ModInt{7}(5)

Hi. I see you're multiplying with an Int32. Good, because I have a better algorithm for this case!


ModInt{7}(3)

This was a silly example, but in case you *do* have a better example for a specific case, it is trivial to add it. Have a new structured matrix with a fast matrix-vector product? Define a type for the matrix, and define multiplication with a vector. That's it: your fast matrix-vector product will be used whenever `A*x` comes up. Hey, that's exactly what we will do in the next section! But first a final comment on dispatch.

### Ambiguity warnings: what if different signatures match your argument types?

In case there is ambiguity, the compiler will emit a warning. This often happens when duck-typing, i.e., if you don't specify the type of an argument. An untyped argument will match any type. For the purposes of dispatch, an untyped argument has type `Any`.

Here's an example:

In [30]:
combine(a, b::ModInt{7}) = "method 1"

combine (generic function with 1 method)

In [31]:
combine(a::ModInt{7}, b) = "method 2"

    combine(Main.ModInt{7}, Any) at In[31]:1
is ambiguous with: 
    combine(Any, Main.ModInt{7}) at In[30]:1.
To fix, define 
    combine(Main.ModInt{7}, Main.ModInt{7})
before the new definition.


combine (generic function with 2 methods)

There is a warning (the first time you evaluate the code) because both type signatures match when `a` and `b` are both of type `ModInt`. There is no rule to prefer one method over the other, because neither is more specific than the other. If this case never happens at runtime then it will never be a problem. The compiler is simply saying it *could* happen. If it *does* happen, Julia will just pick one method.

In [32]:
combine(ModInt{7}(2), ModInt{7}(3))

"method 1"

The solution in this case is to do as the compiler suggested in the warning message: provide a definition for the more specific case. Since this definition is more specific, it takes precedence over the other two and the ambiguity is resolved:

In [33]:
combine(a::ModInt{7}, b::ModInt{7}) = "method 3"

combine (generic function with 3 methods)

In [34]:
combine(ModInt{7}(2), ModInt{7}(3))

"method 3"

You'll encounter many ambiguity warnings soon enough. It is not always as simple to get rid of them, especially if one of your methods clashes with a method in somebody else's code.

If you want to see what I mean, try the same definition of `ModInt` above, but make it a subtype of `Integer`. Now define some operations and try to get rid of all warnings. Good luck with that! (Hint: I've mentioned it before, but look into the [conversion-promotion](http://julia.readthedocs.org/en/release-0.4/manual/conversion-and-promotion/) system. Also, ask yourself: of course a `ModInt` is an integer, but is that sufficient reason to actually inherit from `Integer`?)

## 2. Single dispatch versus multiple dispatch

### Polymorphic member functions in OOP are like single dispatch

Say in an object-oriented language you have an abstract or virtual `Matrix` class, and two subclasses called `DenseMatrix` and `LowRankMatrix`. Perhaps the `Matrix` class has a member function called `rank`, which returns the rank of the matrix. You would supply a definition of `rank` for `DenseMatrix` based on the singular value decomposition, for example, while the `LowRankMatrix` probably stores its rank as part of its representation and so `rank` would have a more efficient implementation than the general one.

In any case, for an object called `mat` that is an instance of `Matrix`, you would probably write
```
mat.rank()
```
or
```
mat->rank()
```
and the version of `rank` that is invoked depends on the actual type of `mat` at run-time. This is **polymorphism**.

In Julia you would typically write
```
rank(mat)
```
where the version of `rank` that is invoked depends on the actual type of `mat` at run-time. This, too, is polymorphism. It means exactly the same thing. It is just different syntax.

### An example: low-rank matrices

Our examples are becoming more advanced as we go along. Just for fun, here is a simple definition of a low rank matrix with a fast matrix-vector product.

In [35]:
import Base: *, size, rank

# A LowRankMatrix is an MxN matrix of the form A*B^t, where A is MxK and B is NxK.
# K is the rank of the matrix.
# It is good practice to include the element type as a type parameter: this makes
# the types of a and b concrete (and therefore efficient), but still general.
immutable LowRankMatrix{T}
    a :: Array{T,2}
    b :: Array{T,2}
end

size(A::LowRankMatrix) = ( size(A.a,1), size(A.b,1) )

*{T}(A::LowRankMatrix{T}, x::Array{T,1}) = A.a * A.b' * x

rank(A::LowRankMatrix) = size(A.a, 2)

rank (generic function with 5 methods)

Did you notice how short this code is? We leave it as an exercise to add proper dimensionality checks, for example in an inner constructor and using [assertions](http://julia.readthedocs.org/en/release-0.4/manual/metaprogramming/#building-an-advanced-macro). The code is also not optimal, because the matrix-vector product allocates memory. We will fix that later, when we discuss in-place methods. Still, the matrix-vector product has the right computational complexity: $O((N+M)K)$ instead of $O(NM)$ for multiplication with a dense $M \times N$ matrix.

In [36]:
A = LowRankMatrix(rand(10,2), rand(10,2))
size(A)

(10,10)

In [37]:
rank(A)

2

In [38]:
A*rand(10)

10-element Array{Float64,1}:
 2.79157 
 1.48506 
 1.65913 
 1.59395 
 2.58245 
 1.31322 
 1.84991 
 1.57254 
 0.819181
 1.83003 

### Arithmetics with low-rank matrices

It may make sense to you to have `rank` be a member function of `mat`. Clearly, the rank of a matrix is an inherent property of that matrix. But things start to look differently if you mix several objects. What about addition?

In the OOP statement
```
mat.add(mat2)
```
`mat` is more special than `mat2`, whereas in the Julia formulation
```
add(mat, mat2)
```
their roles are entirely equivalent. It is quite clear that the best algorithm to add two matrices is determined by the types of both matrices, not just the first one. Their order should not matter.

In several typed languages, using function overloading you can define addition for two matrices outside of a class definition. That is how it works in Julia too, using multiple dispatch.

In [39]:
import Base: +

# We simply concatenate to create a matrix of rank k1+k2, where k1=rank(mat1) and k2=rank(mat2)
+(mat1::LowRankMatrix, mat2::LowRankMatrix) = LowRankMatrix([mat1.a mat2.a], [mat2.b mat2.b])

+ (generic function with 157 methods)

In [40]:
A = LowRankMatrix(rand(10,2), rand(10,2))
B = LowRankMatrix(rand(10,2), rand(10,2))
C = A+B
rank(C)

4

We can easily interact with the outside world and express that a low rank matrix times any other matrix is always a low rank matrix.

In [41]:
import Base: *

*(mat1::LowRankMatrix, mat2::AbstractMatrix) = LowRankMatrix(mat1.a, (mat1.b'*mat2)')

* (generic function with 142 methods)

And then we can create a more specific way to multiply low rank matrices. Though the more general routine above would work for this case too, the definition below is *slightly* more efficient because it uses fewer transposes in total. Since we know this special case, we plug it in, and Julia will make sure it is used on every occasion.

In [42]:
*(mat1::LowRankMatrix, mat2::LowRankMatrix) = LowRankMatrix(mat1.a, mat1.b*mat2.a'*mat2.b)

* (generic function with 143 methods)

In [43]:
@which A*rand(10,10)

*(mat1::LowRankMatrix{T}, mat2::AbstractArray{T,2}) at In[41]:3

In [44]:
@which A*B

*(mat1::LowRankMatrix{T}, mat2::LowRankMatrix{T}) at In[42]:1

In [45]:
rank(A*B)

2

## 3. Static versus dynamic multiple dispatch

### Dynamic means things can change

A language like C++ heavily supports *generic programming* with templates and function overloading. Yet, all of that is static: all types are known at compile-time.

Julia is dynamic, and this has some advantages. Here is an example, going back to integers modulo a number `N`. Remember we had an awkward constructor before. We had to fully qualify the type parameters and write `ModInt{7}(2)`. But we can also define this function:

In [46]:
modint(n, N) = ModInt{N}(n)

modint (generic function with 1 method)

In [47]:
a = modint(2, 7)

ModInt{7}(2)

In [48]:
typeof(a)

ModInt{7}

The function `modint` creates a new integer modulo `N`. Though `N` is a type parameter for `ModInt`, it is just an argument to the `modint` function. For all we know, its value could come from user input. There is no way to statically determine what the full type of the `ModInt` is going to be.

Strictly speaking, this makes `modint` type-unstable (if that is even a word). The type of the output depends on a value of the input, not just on the input types. Julia will not know at compile-time what the type of the output will be.

In [49]:
@code_typed modint(Int64,Int64)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:n,:N], Any[Any[Any[:n,Type{Int64},0],Any[:N,Type{Int64},0]],Any[],Any[],Any[]], :(begin  # In[46], line 1:
        return $(Expr(:new, ModInt{Int64}, :((top(convert))(Main.Int64,(Main.mod)(n::Type{Int64},Int64)))))
    end::ModInt{Int64}))))

See the last line, where it says `end::ModInt{Int64}`. The compiler can deduce that `N` is going to be an integer, but it does not know which one it is. Compare this to:

In [50]:
@code_typed ModInt{7}(2)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[symbol("#s41"),:n], Any[Any[Any[symbol("#s41"),Type{ModInt{7}},0],Any[:n,Int64,0]],Any[],Any[],Any[:N]], :(begin  # In[13], line 4:
        return $(Expr(:new, ModInt{7}, :((Base.box)(Int64,(Base.smod_int)(n::Int64,7))::Int64)))
    end::ModInt{7}))))

Here, the compiler has full knowledge about the type at the end.

Our `modint` function is convenient: we can create `ModInt`'s on the fly for any `N` the user would like. But type-inference has a problem tracking the flow of types through the code. I've warned against type instabilities. Is this going to be a problem? Well, yes and no. *One type instability does not a disaster make.* Read on.

### The effect of type-instability is always local: dynamic dispatch helps type inference recover

That is one complicated title, but here is the thing, and it is rather fundamental. Perhaps Julia can not infer the type of one of your variables. Pity, expect things to slow down. Yet, as soon as you invoke a function, dynamic dispatch guarantees to call the most specific method available. Julia checks the types of the arguments and compiles a version of the method specifically for those types. If necessary, this happens at runtime. **Aha!** But that means that **inside the function we've called, the compiler knows all the types again** during compilation! Type inference can continue on its merry way.

This definitely calls for an example, because you don't have to take my word for it. Let's have a type with an untyped field, which from its name we expect to be an array of some kind.

In [51]:
type VerySlowType
    some_array
end

Perhaps we want to sum over the elements in `some_array`. Here is one way to do it. Admittedly it is a strange way (see also our earlier discussion of sums in the previous chapter) but that is not the point:

In [52]:
import Base: sum

function sum(v::VerySlowType)
    z = v.some_array[1]
    for i = 2:length(v.some_array)
        z = z + v.some_array[i]
    end
    z
end

sum (generic function with 13 methods)

In [53]:
v = VerySlowType( rand(10000))
typeof(v.some_array)

Array{Float64,1}

In [54]:
sum(v)
@time sum(v)

4988.0690973054125

0.001421 seconds (49.13 k allocations: 931.698 KB)


Why is the summation so slow? Julia has JIT-compiled `sum` for arguments of type `VerySlowType`. It has no way of knowing what the type of `v.slow_array` will be, so it emits general and slow code that can deal with any type. That is what is going on in the for loop. It is not optimized for arrays of `Float64`'s.

The Julia miracle is that one indirection completely solves this problem:

In [55]:
sum2(v::VerySlowType) = sum_some_array(v.some_array)

function sum_some_array(some_array)
    z = some_array[1]
    for i = 2:length(some_array)
        z = z + some_array[i]
    end
    z
end

sum_some_array (generic function with 1 method)

In [56]:
v = VerySlowType( rand(10000))
sum2(v)
@time sum2(v)

5020.159448504227

0.000011 seconds (5 allocations: 176 bytes)


There you are: blazingly fast and zero allocations! With almost exactly the same source code. Why? **Because `sum_some_array` knows the type of `some_array`, even though `sum2` does not!** The call to `sum_some_array` from `sum2` incurs a dynamic performance hit because the type of `v.some_array` is unknown. But this type will be determined and Julia sees it is an array of `Float64`'s. It will call a version of `sum_some_array` that is optimized for arrays of `Float64`'s.

There is a one-time performance hit for compiling `sum_some_array`. There is also a recurring performance hit for looking up the type of `some_array` at runtime, each time we invoke `sum2`, because we call another function with `some_array` as an argument. But that is limited to just one function call, and the time it takes to resolve is dwarfed by the execution time of the sum. The important thing to optimize here is the summation itself, and that is what with a single indirection we have achieved.

This is perhaps a subtle point to make, but it is a very important one. Go back to the example until you've convinced yourself that you've completely understood. It brings you that much closer to appreciating where the speed of Julia comes from and how you can write efficient code.

## 4. Some design patterns with dispatch

### Data or metadata: do I use a type parameter or an extra field?

Remember the `ModInt` type I defined earlier in this chapter, for integers modulo `N`? It had the number `N` as a type parameter. It exists only in the compiler. I could also have used this definition:

In [57]:
immutable ModInt2
    n :: Int64
    N :: Int64
    
    ModInt2(n, N) = new(mod(n,N), N)
end

There are advantages and disadvantages. With the second definition, all numbers would take twice as much memory:

In [58]:
sizeof(ModInt2(5, 7))

16

In [59]:
sizeof(ModInt{7}(5))

8

On the other hand, I could efficiently use thousands of different values of `N` in one program. When `N` is a type parameter as in `ModInt{N}`, each different value of `N` results in a different type. This means all functions and methods that use `ModInt{N}` will be recompiled, adding to compilation overhead and size of code.

Another consideration: when `N` is a type parameter, I can dispatch on the value of `N`. Not so when `N` is a field, because you can not dispatch on the value of a variable, only on its type. I would be forced to write things like:

In [60]:
function +(a::ModInt2, b::ModInt2)
    if a.N == b.N
        ModInt2(a+b, a.N)
    else
        error("No can do.")
    end
end

+ (generic function with 158 methods)

Such kinds of `if` statements at the start of a function are a clear indication that dispatch may be a more elegant alternative.

There is a trade-off between speed of execution and compilation time. In the case of integers modulo `N`, having a type parameter `N` seems like the most defensible option. But a different choice was made for the sizes of vectors and matrices. Julia includes the dimension of an array as a type parameter, but not its actual size:

In [61]:
a = Array(Float64, 10,10)
typeof(a)

Array{Float64,2}

In [62]:
size(a)

(10,10)

Imagine the amount of recompilation if each function using matrices depended on the size of the matrix! Usually, the cost associated with verifying correct dimensions is dwarfed by the cost of the actual operation on the matrix anyway. Removing the `if` statement will not make much difference.

The overhead may be significant for small matrices though. And, sure enough, there is a packaged called [`FixedSizeArrays`](https://github.com/SimonDanisch/FixedSizeArrays.jl) which includes the size of an array in its type and leads to better performance for small arrays.

### Efficient method selection with singleton types

You often have several options for solving a particular numerical problem. It is customary in other languages to provide options with integer arguments, or even strings. An example could be `dwt(x, L, "db2")` to compute the discrete wavelet transform of a vector `x` to `L` levels using the Daubechies 2 wavelet ("db2").

Singleton types carry no data, but their type is useful metadata. They can be used to select methods.

In [1]:
function dwt(x, L, name)
    if name == "db2"
        "Computing a wavelet transform using Daubechies 2"
    elseif name == "db4"
        "Computing a wavelet transform using Daubechies 4"
    else
        "Using a generic algorithm for some general wavelet"
    end
end

dwt (generic function with 1 method)

In [2]:
abstract DiscreteWavelet

immutable Daubechies{N} <: DiscreteWavelet
end

immutable SomeOtherWavelet <: DiscreteWavelet
end

In [3]:
dwt{T <: DiscreteWavelet}(x, L, ::Type{T}) = "Using a generic algorithm for some general wavelet"

dwt{N}(x, L, ::Type{Daubechies{N}}) = "Computing a wavelet transform using Daubechies $N"

dwt (generic function with 3 methods)

The first method of `dwt` illustrates how to write code that applies to any wavelet: the type parameter `T` is restricted to be a subtype of DiscreteWavelet.

The second method may use a more specific algorithm for a Daubechies wavelet.

In [5]:
dwt(rand(10), 4, SomeOtherWavelet)

"Using a generic algorithm for some general wavelet"

In [6]:
dwt(rand(10), 4, Daubechies{2})

"Computing a wavelet transform using Daubechies 2"

One important thing to remark is that option selection using singleton types makes your code type-stable. Each different singleton type yields a different type signature of the method. So, it is okay if the output type is different, for example due to using a different algorithm. (That was not the case above because I simply returned strings, but you get the point.)

Other advantages include:
* increased probability of inlining: no list of conditional statements
* ease of adding options: you don't have to modify the `if elseif` list in `dwt`, you simply define a new wavelet and add a method for it
* improved performance: your code jumps immediately to the right function.

### Is value-based dispatch a good idea?

Once you're used to dispatch based on types, you may become greedy and notice some of your code would become faster if you could also dispatch on values. It may allow you to remove even more `if` and `case` statements.

Tracking values of variables is orders of magnitude harder than tracking types. In Julia, I don't think it will happen. The performance overhead for analyzing possible values a variable might have would be immense. The only advantage seems to be that you can more easily add special cases for special values, for example to define the limit of a function at a point.

One thing that is provided is the [`Val{T}`](http://docs.julialang.org/en/release-0.4/manual/types/#value-types) type. The type parameter `T` can be any [*bitstype*](http://docs.julialang.org/en/release-0.4/manual/types/#bits-types), including `Float64`, `Bool`, `Int64` and so on. Here, the compiler knows the numerical value of `T`, because it is part of the type `Val{T}`. You can do computations with `T` at compile-time. The trade-off is that each numerical value for `T` results in a new type and, hence, in more code and compilation time. Actually, `Val{T}` is just a singleton for each `T`, and so the above comments on singleton types and parameter-versus-field apply.

Here is an example where `Val{T}` seems to make good sense. Let's say you want a routine for computing the $l^p$ norm of a vector. You could write:

In [87]:
function norm(x, p)
    if p == 1
        sum(abs(x))
    elseif p == Inf
        maximum(abs(x))
    elseif p == 0
        countnz(x)  ## number of non-zero entries
    else
        sum(abs(x).^p)^(1/p)
    end
end

norm (generic function with 4 methods)

The conditional statements make this routine less readable, and more difficult to inline. However, you are unlikely to use many different values of $p$. The important ones are mainly 0, 1, 2 and $\infty$. Here is a more elegant solution.

In [7]:
# general definition
norm{P}(x, ::Type{Val{P}}) = sum(abs(x).^P).^(1/P)

# l0 norm: count number of non-zero entries
norm(x, ::Type{Val{0}}) = countnz(x)

# l1 norm: sum of absolute values
norm(x, ::Type{Val{1}}) = sum(abs(x))

# l∞ norm: maximum of absolute value
norm(x, ::Type{Val{Inf}}) = maximum(abs(x))

norm (generic function with 4 methods)

In [8]:
a = rand(10)
norm(a, Val{2})

1.6064764819798238

In [9]:
norm(a, Val{1})

4.156542611623095

In [10]:
norm(a, Val{Inf})

0.8642885855908229

In [11]:
@which norm(a, Val{1})

norm(x, ::Type{Val{1}}) at In[7]:8

In [12]:
@which norm(a, Val{Inf})

norm(x, ::Type{Val{Inf}}) at In[7]:11

In [13]:
@which norm(a, Val{3})

norm{P}(x, ::Type{Val{P}}) at In[7]:2